In [1]:
import enum
import itertools
import struct
import typing

In [2]:
# because running the notebook in VSCode makes the working dir the notebook root,
# not where the notebook's located...
import os
import pathlib
os.chdir(pathlib.Path("~/Code/control-freak").expanduser())

In [3]:
with open("dumps/CMC850.FA1.4", mode="rb") as f:
    data = f.read()
    
len(data)

8192

In [4]:
RECIPE_SIZE = 36  # inferred from looking at data
SUBGROUP_SIZE = 256
RECIPES_PER_SUBGROUP = int(SUBGROUP_SIZE / RECIPE_SIZE)

def chunks(iterable, size):
    it = iter(iterable)
    item = list(itertools.islice(it, size))
    while item:
        yield item
        item = list(itertools.islice(it, size))

def extract_entries(data):
    for subgroup in chunks(data, SUBGROUP_SIZE):
        entries = chunks(subgroup, RECIPE_SIZE)
        for entry in itertools.islice(entries, RECIPES_PER_SUBGROUP):
            entry = bytes(entry)
            if entry.startswith(b"\xFF"):
                return
            yield entry

entries = list(extract_entries(data))
len(entries)

23

# Entries

36 bytes, repeating.

* Bytes 0-x: `char[]`, name. ASCII name at start, max varies, around 19-20 chars? (`0123456789abcdefghi` and `Llllllllllllllllllll` capped it, char width max on display?)
* Byte 30 (1E): LSByte of temperature in °F? Device can set temp from 77°F to 482°F (406 set points).

```
       b30  b31[7]
 77 ->  77   0
155 -> 155   0
156 -> 156   0
254 -> 254   0
255 -> 255   0
256 ->   1   1
257 ->   2   1
258 ->   3   1
482 -> 227   1
```
* Byte 31 (1F): 
    * bit 7: +255 to temp?
    * bits 6-5: action at timer end
        * `00`: Continue
        * `01`: Stop
        * `10`: Keep Warm 140°F/60°C
        * `11`: Repeat (NEED TO CONFIRM)
    * bit 4: ???
    * bits 3-2: intensity/speed
        * `00`: Slow
        * `01`: Medium
        * `10`: Fast
        * `11`: unknown/undefined?
    * bits 1-0: timer start condition
        * `00`: At Beginning
        * `01`: At Temp
        * `10`: At Prompt
        * `11`: unknown/undefined?
* Byte 32 (20): `int8`, hours. Device caps out at 72 hours.
* Byte 33 (21): `int8`, minutes.
* Byte 34 (22): `int8`, seconds. Device only allows selecting multiples of 5.
* Byte 35 (23): `char`, Checksum, 8-bit sum of previous 35 bytes

In [5]:
class TimerStart(enum.Enum):
    BEGINNING = "beginning"
    AT_TEMP = "at_temp"
    PROMPT = "prompt"

class TimerEnd(enum.Enum):
    STOP = "Stop Cooking"
    CONTINUE = "Continue"
    KEEP_WARM = "Keep Warm"
    REPEAT = "Repeat"
    
class Speed(enum.Enum):
    FAST = "Fast"
    MEDIUM = "Medium"
    SLOW = "Slow"

class Entry(typing.NamedTuple):
    name: str
    hours: int
    minutes: int
    seconds: int
    temp_f: int
    speed: Speed
    timer_start: TimerStart
    timer_end: TimerEnd
    checksum: bool
#     raw: bytes

In [6]:
def parse_name(entry):
    return entry[:20].split(b"\x00", 1)[0].decode("ascii")
    
def parse_temp(entry):
    t_lsb = entry[0x1E]
    t_extra = entry[0x1F] & 0b1000_0000
    if t_extra:
        return t_lsb + 255
    return t_lsb

def parse_timer_start(entry):
    b = entry[0x1F]
    timer_bits = (b & 0b0000_0011) >> 0
    return {
        0b00: TimerStart.BEGINNING,
        0b01: TimerStart.AT_TEMP,
        0b10: TimerStart.PROMPT,
    }[timer_bits]

def parse_timer_end(entry):
    b = entry[0x1F]
    timer_bits = (b & 0b0110_0000) >> 5
#     return timer_bits
    return {
        0b00: TimerEnd.CONTINUE,
        0b01: TimerEnd.STOP,
        0b10: TimerEnd.KEEP_WARM,
        0b11: TimerEnd.REPEAT,  # UNCONFIRMED
    }[timer_bits]

def parse_speed(entry):
    b = entry[0x1F]
    temp_bits = (b & 0b0000_1100) >> 2
    return {
        0b00: Speed.SLOW,
        0b01: Speed.MEDIUM,
        0b10: Speed.FAST,
    }[temp_bits]

def parse_checksum(entry):
    computed = sum(entry[0:35]) % 256
    return computed == entry[35]

def parse_entry(b: bytes):
    assert len(b) == RECIPE_SIZE
    
    return Entry(
#         raw=b,
        name=parse_name(b),
        hours=b[32],
        minutes=b[33],
        seconds=b[34],
        temp_f=parse_temp(b),
        timer_start=parse_timer_start(b),
        timer_end=parse_timer_end(b),
        speed=parse_speed(b),
        checksum=parse_checksum(b),
    )

In [7]:
def viz_entry(e: Entry):
    print(e.name)
    print(f" - {e.temp_f}\N{DEGREE SIGN}F")
    print(" -", e.speed.value.capitalize())
    if e.hours:
        print(f" - {e.hours}:{e.minutes:02d}HRS")
    else:
        print(f" - {e.minutes}:{e.seconds:02d}MIN")
#     print(" -", e.timer_end.value)
    print(" -", e.timer_end)

In [8]:
viz_entry(parse_entry(b"Llllllllllllllllllll\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xe3\xc0H\x00\x00;"))
# keep warm

Llllllllllllllllllll
 - 482°F
 - Slow
 - 72:00HRS
 - TimerEnd.KEEP_WARM


In [9]:
viz_entry(parse_entry(b"Poaching\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x9b%\x00-\x00\x16"))
# stop

Poaching
 - 155°F
 - Medium
 - 45:00MIN
 - TimerEnd.STOP


In [10]:
viz_entry(parse_entry(b"77pfc\x001\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00M\n\x00\x00\x054"))
# continue

77pfc
 - 77°F
 - Fast
 - 0:05MIN
 - TimerEnd.CONTINUE


In [11]:
for e in entries:
    viz_entry(parse_entry(e))

Temp 01
 - 250°F
 - Fast
 - 0:00MIN
 - TimerEnd.CONTINUE
0                  0
 - 260°F
 - Fast
 - 0:05MIN
 - TimerEnd.REPEAT
0123456789abcdefghi
 - 257°F
 - Fast
 - 1:00MIN
 - TimerEnd.REPEAT
1
 - 251°F
 - Fast
 - 1:00MIN
 - TimerEnd.REPEAT
2
 - 252°F
 - Fast
 - 0:05MIN
 - TimerEnd.REPEAT
255sfc
 - 255°F
 - Fast
 - 0:05MIN
 - TimerEnd.CONTINUE
256sfc
 - 256°F
 - Fast
 - 0:05MIN
 - TimerEnd.CONTINUE
333
 - 253°F
 - Fast
 - 0:05MIN
 - TimerEnd.CONTINUE
4444
 - 254°F
 - Fast
 - 0:05MIN
 - TimerEnd.CONTINUE
55555
 - 255°F
 - Fast
 - 0:05MIN
 - TimerEnd.CONTINUE
666666
 - 256°F
 - Fast
 - 0:05MIN
 - TimerEnd.CONTINUE
7777777
 - 257°F
 - Fast
 - 0:05MIN
 - TimerEnd.CONTINUE
77pfc
 - 77°F
 - Fast
 - 0:05MIN
 - TimerEnd.CONTINUE
77pfs
 - 77°F
 - Fast
 - 0:05MIN
 - TimerEnd.STOP
77sfc
 - 77°F
 - Fast
 - 0:05MIN
 - TimerEnd.CONTINUE
77smc
 - 77°F
 - Medium
 - 0:05MIN
 - TimerEnd.CONTINUE
77ssc
 - 77°F
 - Slow
 - 0:05MIN
 - TimerEnd.CONTINUE
77tfc
 - 77°F
 - Fast
 - 0:05MIN
 - TimerEnd.CONTINUE
8

In [12]:
e3 = parse_entry(entries[3])
e3.name #, e3.raw[0x1F]

'1'

In [13]:
parse_entry(entries[0])

Entry(name='Temp 01', hours=0, minutes=0, seconds=0, temp_f=250, speed=<Speed.FAST: 'Fast'>, timer_start=<TimerStart.BEGINNING: 'beginning'>, timer_end=<TimerEnd.CONTINUE: 'Continue'>, checksum=True)

In [14]:
parse_entry(entries[1])

Entry(name='0                  0', hours=0, minutes=0, seconds=5, temp_f=260, speed=<Speed.FAST: 'Fast'>, timer_start=<TimerStart.PROMPT: 'prompt'>, timer_end=<TimerEnd.REPEAT: 'Repeat'>, checksum=True)

In [15]:
parse_entry(entries[2])

Entry(name='0123456789abcdefghi', hours=0, minutes=1, seconds=0, temp_f=257, speed=<Speed.FAST: 'Fast'>, timer_start=<TimerStart.BEGINNING: 'beginning'>, timer_end=<TimerEnd.REPEAT: 'Repeat'>, checksum=True)

In [16]:
parse_entry(entries[4])

Entry(name='2', hours=0, minutes=0, seconds=5, temp_f=252, speed=<Speed.FAST: 'Fast'>, timer_start=<TimerStart.BEGINNING: 'beginning'>, timer_end=<TimerEnd.REPEAT: 'Repeat'>, checksum=True)

In [17]:
parse_entry(entries[6])

Entry(name='256sfc', hours=0, minutes=0, seconds=5, temp_f=256, speed=<Speed.FAST: 'Fast'>, timer_start=<TimerStart.BEGINNING: 'beginning'>, timer_end=<TimerEnd.CONTINUE: 'Continue'>, checksum=True)